In [607]:
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.stats as stats
from pickle import load
from scipy import stats
import pickle

In [608]:
def create_child(mother, father, beta, mutation_rate, window_size):

    child = np.add(beta*father, (1-beta)*mother)

    entries_mutate = random.sample(range(0, len(child)), int(mutation_rate*len(child)))

    for i in range(0,int(mutation_rate*len(child))):
        bottom_window = entries_mutate[i] - window_size
        top_window = entries_mutate[i] + window_size
        if bottom_window >= 0 and top_window <= len(child):
            sliding_window = child[bottom_window:top_window]
            mu, sigma = sliding_window.mean(), sliding_window.var()
            lower, upper = 0, 1
            truncated_normal = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)

            child[entries_mutate[i]] = truncated_normal.rvs(1)[0]

        elif bottom_window < 0:
            sliding_window = child[0:top_window]
            mu, sigma = sliding_window.mean(), sliding_window.var()
            lower, upper = 0, 1
            truncated_normal = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)

            child[entries_mutate[i]] = truncated_normal.rvs(1)[0]
        
        elif top_window > len(child):
            sliding_window = child[bottom_window:len(child)]
            mu, sigma = sliding_window.mean(), sliding_window.var()
            lower, upper = 0, 1
            truncated_normal = stats.truncnorm((lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)

            child[entries_mutate[i]] = truncated_normal.rvs(1)[0]
    
    return child

In [609]:
def one_generation(population, number_to_keep, beta, mutation_rate, window_size):

    parents = population[0:number_to_keep]
    number_of_children = len(population) - len(parents)

    random_pairs = random.sample(range(0, len(parents)), len(parents))

    children = []
    k = 0 

    while len(children) < number_of_children:
        child = create_child(parents[random_pairs[k]][0], parents[random_pairs[k+1]][0], beta, mutation_rate,window_size)
        children.append([child, 0])
        # I changed -1 to -2
        if k == len(random_pairs)-2:
            k = 0
        else:
            k += 2

    generation = []

    generation.extend(parents)
    generation.extend(children)

    return generation

In [610]:
def takeSecond(elem):
    return elem[1]

def genetic_algorithm(model, initial_population, generations, number_to_keep, beta, mutation_rate, window_size):

    evolved_population = []
    evolving_population = []

    for pop in initial_population:
            evolving_population.append([pop, model.predict_proba(pop.reshape(1,-1))[0][1]])

    for x in range(0, generations):

        for pop in evolved_population:
            evolving_population.append([pop[0], model.predict_proba(pop[0].reshape(1,-1))[0][1]])
        
        evolving_population.sort(key=takeSecond, reverse=True)

        evolved_population = one_generation(evolving_population, number_to_keep, beta, mutation_rate, window_size)

        evolving_population = []

    for pop in evolved_population:
            evolving_population.append([pop[0], model.predict_proba(pop[0].reshape(1,-1))[0][1]])

    return evolving_population

In [611]:
# Some helper functions

def images_probabilities(data):

    images = []
    probabilities = []

    for i in range(0,len(data)):
        images.append(data[:][i][0])
        probabilities.append(data[:][i][1])
    
    return images, probabilities

def probabilities(data, model):
    probabilities = []
    
    for x in data:
        temp = model.predict_proba(x.reshape(1,-1))[0][1]
        probabilities.append(temp)
        
    return probabilities

def kl_divergence(p, q):
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))
    

In [ ]:
# Set global parameters
generations = 30
number_to_keep = 90
beta = 0.5
mutation_rate = 0.95
window_size = 2


# Import the data and model
model = load(open('mlp.pkl', 'rb'))
AU_Face_Data = np.load('AU_Faces.npy')
EU_Face_Data = np.load('EU_Faces.npy')

# Randomly sample an initial population
#AU_Initial_Population = select_population(AU_Face_Data, pop_number)
AU_Initial_Population = list(AU_Face_Data)
AU_Unevolved_Probabilities = probabilities(AU_Initial_Population, model)
EU_Initial_Population = list(EU_Face_Data)
EU_Unevolved_Probabilities = probabilities(EU_Initial_Population, model)


# Conduct mutation and return the evolved samples
AU_Evolved = genetic_algorithm(model, AU_Initial_Population, generations, number_to_keep, beta, mutation_rate, window_size)
AU_Evolved_Images, AU_Evolved_Probabilities = images_probabilities(AU_Evolved)

np.save("AU_Evolved", AU_Evolved_Images)

# Conduct mutation and return the evolved samples

EU_Evolved = genetic_algorithm(model, EU_Initial_Population, generations, number_to_keep, beta, mutation_rate, window_size)
EU_Evolved_Images, EU_Evolved_Probabilities = images_probabilities(EU_Evolved)

np.save("EU_Evolved", EU_Evolved_Images)

print("evolutionary process complete")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


In [ ]:
#T-Test of different populations

print(f'AU Initial + AU Evolved p-value: {stats.ttest_ind(AU_Unevolved_Probabilities, AU_Evolved_Probabilities)[1]}')
print(f'EU Initial + EU Evolved p-value: {stats.ttest_ind(EU_Unevolved_Probabilities, EU_Evolved_Probabilities)[1]}')
print(f'EU Initial + AU Initial p-value: {stats.ttest_ind(AU_Unevolved_Probabilities, EU_Unevolved_Probabilities)[1]}')
print(f'AU Evolved + EU Evolved p-value: {stats.ttest_ind(AU_Evolved_Probabilities, EU_Evolved_Probabilities)[1]}')

print(f'AU Initial + AU Evolved T-Stat: {stats.ttest_ind(AU_Unevolved_Probabilities, AU_Evolved_Probabilities)[0]}')
print(f'EU Initial + EU Evolved T-Stat: {stats.ttest_ind(EU_Unevolved_Probabilities, EU_Evolved_Probabilities)[0]}')
print(f'EU Initial + AU Initial T-Stat: {stats.ttest_ind(AU_Unevolved_Probabilities, EU_Unevolved_Probabilities)[0]}')
print(f'AU Evolved + EU Evolved T-Stat: {stats.ttest_ind(AU_Evolved_Probabilities, EU_Evolved_Probabilities)[0]}')


In [ ]:
#Avergae Performance AU
print(f"AU_EVOLVED_PROBABILITIES : {np.average(AU_Evolved_Probabilities)}")
print(f"AU_INITIAL_PROBABILITIES : {np.average(AU_Unevolved_Probabilities)}")


In [ ]:
#Avergae Performance EU
print(f"EU_EVOLVED_PROBABILITIES : {np.average(EU_Evolved_Probabilities)}")
print(f"EU_INITIAL_PROBABILITIES : {np.average(EU_Unevolved_Probabilities)}")

In [ ]:
labels = ['Initial', 'Evolved']
EU_means = [np.average(EU_Unevolved_Probabilities), np.average(EU_Evolved_Probabilities)]
AU_means = [np.average(AU_Unevolved_Probabilities), np.average(AU_Evolved_Probabilities)]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, EU_means, width, label='EU')
rects2 = ax.bar(x + width/2, AU_means, width, label='AU')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Average Probability')
ax.set_title('Performance by evolution and group')
ax.set_xticks(x)
ax.set_ylim(0, 1)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


fig.tight_layout()

fig.savefig("Groups/performance.png")

In [ ]:
# Comparing the two evolved subgroups 

# Define the number of bins 
bins = 256
# Calculate some histograms of the evolved vectors 
AU_evolved_probabilities, AU_hist = np.histogram(np.array(AU_Evolved_Images).ravel(), bins = bins, density = False)
EU_evolved_probabilities, EU_hist = np.histogram(np.array(EU_Evolved_Images).ravel(), bins = bins, density = False)

# Normalise them and use them as proxies for pmfs
AU_evolved_probabilities = AU_probabilities/sum(AU_probabilities)
EU_evolved_probabilities = EU_probabilities/sum(EU_probabilities)
x_axis = np.arange(0,1,1/bins)

# Plot the two graphs 
plt.title('KL(AU Evolved||EU Evolved) = %1.3f' % kl_divergence(AU_evolved_probabilities, EU_evolved_probabilities))
plt.plot(x_axis, AU_evolved_probabilities, c = 'orange')
plt.plot(x_axis, EU_evolved_probabilities, c='black')

In [ ]:
# Comparing the two initial subgroups 

# Define the number of bins 
bins = 256
# Calculate some histograms of the evolved vectors 
AU_initial_probabilities, AU_hist = np.histogram(np.array(AU_Initial_Population).ravel(), bins = bins, density = False)
EU_initial_probabilities, EU_hist = np.histogram(np.array(EU_Initial_Population).ravel(), bins = bins, density = False)

# Normalise them and use them as proxies for pmfs
AU_initial_probabilities = AU_initial_probabilities/sum(AU_initial_probabilities)
EU_initial_probabilities = EU_initial_probabilities/sum(EU_initial_probabilities)
x_axis = np.arange(0,1,1/bins)

# Plot the two graphs 
plt.title('KL(AU Initial||EU Initial) = %1.3f' % kl_divergence(AU_initial_probabilities, EU_initial_probabilities))
plt.plot(x_axis, AU_initial_probabilities, c ='orange')
plt.plot(x_axis, EU_initial_probabilities, c='black')

In [ ]:
# Comparing the AU evolved and initial 

# Define the number of bins 
bins = 256
# Calculate some histograms of the evolved vectors 
AU_initial_probabilities, AU_hist = np.histogram(np.array(AU_Initial_Population).ravel(), bins = bins, density = False)
AU_evolved_probabilities, AU_hist = np.histogram(np.array(AU_Evolved_Images).ravel(), bins = bins, density = False)

# Normalise them and use them as proxies for pmfs
AU_initial_probabilities = AU_initial_probabilities/sum(AU_initial_probabilities)
AU_evolved_probabilities = AU_evolved_probabilities/sum(AU_evolved_probabilities)
x_axis = np.arange(0,1,1/bins)

# Plot the two graphs 
plt.title('KL(AU Initial||AU Evolved) = %1.3f' % kl_divergence(AU_initial_probabilities, AU_evolved_probabilities))
plt.plot(x_axis, AU_initial_probabilities, c = 'blue')
plt.plot(x_axis, AU_evolved_probabilities, c='red')

In [ ]:
# Comparing the eu evolved and initial

# Define the number of bins 
bins = 256
# Calculate some histograms of the evolved vectors 
EU_initial_probabilities, EU_hist = np.histogram(np.array(EU_Initial_Population).ravel(), bins = bins, density = False)
EU_evolved_probabilities, EU_hist = np.histogram(np.array(EU_Evolved_Images).ravel(), bins = bins, density = False)

# Normalise them and use them as proxies for pmfs
EU_initial_probabilities = EU_initial_probabilities/sum(EU_initial_probabilities)
EU_evolved_probabilities = EU_evolved_probabilities/sum(EU_evolved_probabilities)
x_axis = np.arange(0,1,1/bins)

# Plot the two graphs 
plt.title('KL(EU Initial||EU Evolved) = %1.3f' % kl_divergence(EU_initial_probabilities, EU_evolved_probabilities))
plt.plot(x_axis, EU_initial_probabilities, c = 'blue')
plt.plot(x_axis, EU_evolved_probabilities, c='red')


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Initial and Evolved Comparisons: Within Group')
ax1.plot(x_axis, EU_initial_probabilities, c ='blue')
ax1.plot(x_axis, EU_evolved_probabilities, c='red')
ax1.set_title('KL(EU Initial||EU Evolved) \n = %1.3f' % kl_divergence(EU_initial_probabilities, EU_evolved_probabilities), wrap=True)
ax2.plot(x_axis, AU_initial_probabilities, c='blue', label = 'Initial')
ax2.plot(x_axis, AU_evolved_probabilities, c='red', label = 'Evolved')
ax2.legend()
ax2.set_title('KL(AU Initial||AU Evolved) \n = %1.3f' % kl_divergence(AU_initial_probabilities, AU_evolved_probabilities), wrap=True)
fig.tight_layout(pad=2.0)

fig.savefig("Groups/within_group.png")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Initial and Evolved Comparisons: Between Group')
ax1.plot(x_axis, EU_initial_probabilities, c ='blue')
ax1.plot(x_axis, AU_initial_probabilities, c='red')
ax1.set_title('KL(AU Initial||EU Initial) \n = %1.3f' % kl_divergence(AU_initial_probabilities, EU_initial_probabilities), wrap=True)
ax2.plot(x_axis, EU_evolved_probabilities, c='blue', label="EU")
ax2.plot(x_axis, AU_evolved_probabilities, c='red', label="AU")
ax2.legend()
ax2.set_title('KL(AU Evolved||EU Evolved) \n = %1.3f' % kl_divergence(AU_evolved_probabilities, EU_evolved_probabilities), wrap=True)
fig.tight_layout(pad=2.0)

fig.savefig("Groups/between_group.png")

In [ ]:
#plt.imsave("EU_Evolved/hat.png",EU_Evolved_Images[5].reshape(80,64,3))

In [ ]:
for x in range(0,len(EU_Evolved_Images)):
    plt.imsave(f"EU_Evolved/EU_Image_{x}.png",EU_Evolved_Images[x].reshape(80,64,3))

In [ ]:
for x in range(0,len(AU_Evolved_Images)):
    plt.imsave(f"AU_Evolved/AU_Image_{x}.png",AU_Evolved_Images[x].reshape(80,64,3))

In [ ]:
with open("Groups/Output.txt", "w") as text_file:
    
    # P-values 
    print(f'AU Initial + AU Evolved p-value: {stats.ttest_ind(AU_Unevolved_Probabilities, AU_Evolved_Probabilities)[1]}', file=text_file)
    print(f'EU Initial + EU Evolved p-value: {stats.ttest_ind(EU_Unevolved_Probabilities, EU_Evolved_Probabilities)[1]}', file=text_file)
    print(f'EU Initial + AU Initial p-value: {stats.ttest_ind(AU_Unevolved_Probabilities, EU_Unevolved_Probabilities)[1]}', file=text_file)
    print(f'AU Evolved + EU Evolved p-value: {stats.ttest_ind(AU_Evolved_Probabilities, EU_Evolved_Probabilities)[1]}', file=text_file)
    
    print(' ', file = text_file)
    
    # T-Stats 
    print(f'AU Initial + AU Evolved T-Stat: {stats.ttest_ind(AU_Unevolved_Probabilities, AU_Evolved_Probabilities)[0]}', file=text_file)
    print(f'EU Initial + EU Evolved T-Stat: {stats.ttest_ind(EU_Unevolved_Probabilities, EU_Evolved_Probabilities)[0]}', file=text_file)
    print(f'EU Initial + AU Initial T-Stat: {stats.ttest_ind(AU_Unevolved_Probabilities, EU_Unevolved_Probabilities)[0]}', file=text_file)
    print(f'AU Evolved + EU Evolved T-Stat: {stats.ttest_ind(AU_Evolved_Probabilities, EU_Evolved_Probabilities)[0]}', file=text_file)
    
    print(' ', file = text_file)
    #Avergae Performance AU
    print(f"AU_EVOLVED_PROBABILITIES : {np.average(AU_Evolved_Probabilities)}", file=text_file)
    print(f"AU_INITIAL_PROBABILITIES : {np.average(AU_Unevolved_Probabilities)}", file=text_file)
    
    print(' ', file = text_file)
    #Avergae Performance AU
    print(f"EU_EVOLVED_PROBABILITIES : {np.average(EU_Evolved_Probabilities)}", file=text_file)
    print(f"EU_INITIAL_PROBABILITIES : {np.average(EU_Unevolved_Probabilities)}", file=text_file)
    
    print(' ', file = text_file)
    print('###########################', file = text_file)
    print('MODEL PARAMETERS', file = text_file)
    
    print(f'Generations : {generations}', file = text_file)
    print(f'Number to keep : {number_to_keep}', file = text_file)
    print(f'Beta : {beta}', file = text_file)
    print(f'Mutation Rate : {mutation_rate}', file = text_file)
    print(f'Window Size : {window_size}', file = text_file)
